In [1]:
import requests
from bs4 import BeautifulSoup
import lxml.html
from selenium import webdriver
import time
import pandas as pd
import re
from tqdm import tqdm

In [2]:
reviews2 = pd.read_csv("top100&place.csv")

In [3]:
lq = pd.read_csv('bcstores.csv')

In [4]:
reviews2.head(20)

,mnf,name,style,alc,ratings,avg_rating,detail_rating,avg_per_user,number_of_reviews,text_reviews,rev_link
0,Four Winds Brewing Co.,Nectarous,Wild Ale,5.50%,123,4.45,['look: 4.5 | smell: 4.75 | taste: 4.25 | feel...,"['4.36', '4.2', '5', '3.4', '3.9', '4.54', '4....",35,"[""Poured from the can into a teku glass. No vi...",https://www.beeradvocate.com/beer/profile/3218...
1,Driftwood Brewery,Sartori Harvest IPA,IPA - American,7.00%,117,4.40,['look: 4.25 | smell: 4.25 | taste: 4.25 | fee...,"['4.28', '4.3', '4.12', '4.24', '4.41', '4.32'...",42,"[""I've heard great things...now I've tasted th...",https://www.beeradvocate.com/beer/profile/1885...
2,Driftwood Brewery,Fat Tug IPA,IPA - American,7.00%,718,4.27,['look: 3.75 | smell: 4.25 | taste: 4 | feel: ...,"['4.05', '4.31', '4.81', '4.17', '3.94', '3.88...",184,['Superb lacing & pours golden pale in the gla...,https://www.beeradvocate.com/beer/profile/1885...
3,Central City Brewers + Distillers,Red Racer IPA (India Pale Ale),IPA - American,6.50%,1136,4.25,['look: 4 | smell: 3 | taste: 4 | feel: 3 | ov...,"['3.46', '4.21', '3.86', '4.75', '4.5', '3.71'...",499,['Look: Clear dark copper color. Pours a light...,https://www.beeradvocate.com/beer/profile/1032...
4,Driftwood Brewery,Singularity,Stout - Russian Imperial,10.60%,180,4.28,['look: 4.25 | smell: 4.25 | taste: 4.5 | feel...,"['4.43', '4.87', '3.2', '4.1', '3.83', '4.21',...",70,"[""2oz bottle (2012, label says 14.0% ABV as op...",https://www.beeradvocate.com/beer/profile/1885...
5,Driftwood Brewery,Raised By Wolves IPA,IPA - New England,7.00%,79,4.32,['look: 5 | smell: 4.75 | taste: 4.5 | feel: 5...,"['4.69', '3', '4.6', '4.17', '4.24', '4.46', '...",33,['Smells like one of those bags of dried fruit...,https://www.beeradvocate.com/beer/profile/1885...
6,Driftwood Brewery,Naughty Hildegard ESB,Bitter - English Extra Special / Strong Bitter...,6.50%,140,4.25,['look: 4 | smell: 3 | taste: 3 | feel: 4 | ov...,"['3.16', '4.21', '4', '4.29', '4.1', '3.61', '...",47,"[""Clear dark amber color. The head was a light...",https://www.beeradvocate.com/beer/profile/1885...
7,Four Winds Brewing Co.,Juxtapose Wild IPA,IPA - American,6.50%,133,4.20,['look: 4.25 | smell: 4.25 | taste: 4.5 | feel...,"['4.4', '4.19', '3.99', '2.6', '4.13', '4.13',...",41,"[""This is new to me. Poured from a 1 pint can,...",https://www.beeradvocate.com/beer/profile/3218...
8,Central City Brewers + Distillers,Red Racer Imperial IPA,IPA - Imperial,9.00%,190,4.17,['look: 4.25 | smell: 4.25 | taste: 4.25 | fee...,"['4.23', '3.26', '3.08', '4.15', '3.86', '3.4'...",66,"[""Appearance: Clear medium copper, decent carb...",https://www.beeradvocate.com/beer/profile/1032...
9,Driftwood Brewery,Latus,Sour - Flanders Red Ale,7.50%,53,4.30,"['SebD from Canada', 'look: 4 | smell: 4.25 | ...","['4.75', '4.47', '3.89', '4.21', '3.71', '4.46...",18,['Big thanks to FondueVoodoo for helping me ro...,https://www.beeradvocate.com/beer/profile/1885...


In [5]:
reviews2.columns

Index(['mnf', 'name', 'style', 'alc', 'ratings', 'avg_rating', 'detail_rating',
       'avg_per_user', 'number_of_reviews', 'text_reviews', 'rev_link'],
      dtype='object')

In [6]:
reviews2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403 entries, 0 to 402
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   mnf                403 non-null    object 
 1   name               403 non-null    object 
 2   style              403 non-null    object 
 3   alc                403 non-null    object 
 4   ratings            403 non-null    int64  
 5   avg_rating         403 non-null    float64
 6   detail_rating      403 non-null    object 
 7   avg_per_user       403 non-null    object 
 8   number_of_reviews  403 non-null    int64  
 9   text_reviews       403 non-null    object 
 10  rev_link           403 non-null    object 
dtypes: float64(1), int64(2), object(8)
memory usage: 34.8+ KB


In [7]:
lq.name = lq.name.str.lower()

In [8]:
reviews2.name = reviews2.name.str.lower()

In [9]:
reviews2.name.index

RangeIndex(start=0, stop=403, step=1)

In [10]:
shared_name = reviews2.name.apply(lambda x: lq[lq.name.str.lower().str.find(x).ge(0)].name).bfill(axis =1).iloc[:,0]
shared_name

0                 four winds - nectarous dry hopped sour
1                                                    NaN
2                        driftwood brewery - fat tug ipa
3                                                    NaN
4                                                    NaN
                             ...                        
398                                mcewan's - scotch ale
399         yellow dog - chase my tail pale ale tall can
400             yellow dog brewing - high 5 hazy ipa can
401                  yellow dog - play dead ipa tall can
402    yellow dog brewing - squirrel chaser hazy pale...
Name: 0, Length: 403, dtype: object

In [11]:
dj = pd.DataFrame(list(zip(shared_name, reviews2.mnf, reviews2.name)), columns = ['lqbeer', 'mnfbeeravo','beeradvo'])

In [12]:
dj['lqmnf']= dj.lqbeer.astype(str).apply(lambda x: x.split("-")[0])

In [13]:
dj.mnfbeeravo = dj.mnfbeeravo.str.lower()

In [14]:
dj.mnfbeeravo = dj.mnfbeeravo.str.replace("brewing","").str.replace("co.","").str.replace("BG","").str.replace(" ","")

In [15]:
dj.lqmnf = dj.lqmnf.str.replace(" ","")

In [16]:
dj = dj[['lqbeer','beeradvo', 'lqmnf', 'mnfbeeravo']]

In [17]:
dj

,lqbeer,beeradvo,lqmnf,mnfbeeravo
0,four winds - nectarous dry hopped sour,nectarous,fourwinds,fourwinds
1,NaN,sartori harvest ipa,nan,driftwoodbrewery
2,driftwood brewery - fat tug ipa,fat tug ipa,driftwoodbrewery,driftwoodbrewery
3,NaN,red racer ipa (india pale ale),nan,centralcitybrewers+distillers
4,NaN,singularity,nan,driftwoodbrewery
...,...,...,...,...
398,mcewan's - scotch ale,scotch ale,mcewan's,wolfpany
399,yellow dog - chase my tail pale ale tall can,chase my tail pale ale,yellowdog,yellowdog
400,yellow dog brewing - high 5 hazy ipa can,high 5 hazy ipa,yellowdogbrewing,yellowdog
401,yellow dog - play dead ipa tall can,play dead ipa,yellowdog,yellowdog


In [18]:
dj.lqmnf =dj.lqmnf.apply(lambda x: x[:-1] if x[-1]==" " else x)

In [19]:
dj.loc[100:103,'lqmnf'] = '33 acres'

In [20]:
res =[]
for i, j in zip(dj.mnfbeeravo, dj.lqmnf):
    print(i,j)
    m = i.find(j)
    print(m)
    res.append(m)

fourwinds fourwinds
0
driftwoodbrewery nan
-1
driftwoodbrewery driftwoodbrewery
0
centralcitybrewers+distillers nan
-1
driftwoodbrewery nan
-1
driftwoodbrewery driftwoodbrewery
0
driftwoodbrewery nan
-1
fourwinds fourwinds
0
centralcitybrewers+distillers nan
-1
driftwoodbrewery nan
-1
superflux nan
-1
driftwoodbrewery nan
-1
centralcitybrewers+distillers nan
-1
parallel49pany nan
-1
steamworkspany nan
-1
fourwinds nan
-1
driftwoodbrewery nan
-1
yellowdog yellowdog
0
crannógales nan
-1
howesoundinn&pany nan
-1
driftwoodbrewery nan
-1
fourwinds nan
-1
fourwinds nan
-1
fourwinds nan
-1
superflux nan
-1
lighthousepany nan
-1
fourwinds nan
-1
lighthousepany lighthouse
0
boomboxpany nan
-1
superflux nan
-1
phillipspany nan
-1
phillipspany nan
-1
storm nan
-1
yellowdog nan
-1
yellowdog yellowdogbrewing
-1
backntry nan
-1
howesoundinn&pany nan
-1
phillipspany nan
-1
lighthousepany nan
-1
storm nan
-1
superflux nan
-1
driftwoodbrewery nan
-1
superflux nan
-1
persephone nan
-1
fourwinds nan
-1
f

In [21]:
dj['mnfcomp'] =res

In [22]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [23]:
dj[dj.mnfcomp==0]

,lqbeer,beeradvo,lqmnf,mnfbeeravo,mnfcomp
0,four winds - nectarous dry hopped sour,nectarous,fourwinds,fourwinds,0
2,driftwood brewery - fat tug ipa,fat tug ipa,driftwoodbrewery,driftwoodbrewery,0
5,driftwood brewery - raised by wolves ipa tall ...,raised by wolves ipa,driftwoodbrewery,driftwoodbrewery,0
7,four winds - juxtapose wild ipa,juxtapose wild ipa,fourwinds,fourwinds,0
17,yellow dog - play dead ipa tall can,play dead ipa,yellowdog,yellowdog,0
27,lighthouse - shipwreck ipa can,shipwreck ipa,lighthouse,lighthousepany,0
64,parallel 49 - schadenfreude pumpkin oktoberfes...,schadenfreude,parallel49,parallel49pany,0
68,parallel 49 - salty scot sea salted caramel sc...,salty scot,parallel49,parallel49pany,0
88,dageraad - burnabarian tall can,burnabarian,dageraad,dageraad,0
119,barkerville - 52 foot stout,52 foot stout,barkerville,barkerville,0


In [24]:
dj[(dj.mnfcomp!=0) & (dj.lqmnf != 'nan')]

,lqbeer,beeradvo,lqmnf,mnfbeeravo,mnfcomp
34,yellow dog brewing - high 5 hazy ipa can,high 5 hazy ipa,yellowdogbrewing,yellowdog,-1
54,russell brewing - a wee angry scotch ale,angry scotch ale,russellbrewing,russellpany,-1
66,vancouver island - hermannator ice bock tall can,hermannator,vancouverisland,vanverisland,-1
73,fernie - project 9 pilsner can,pilsner,fernie,fourwinds,-1
85,twin sails brewing - dat juice tall can,dat juice,twinsailsbrewing,twinsails,-1
100,33 acres of life lager,33 acres of life,33 acres,33acres,-1
101,33 acres of nirvana 6 pack,33 acres of nirvana,33 acres,33acres,-1
102,33 acres of ocean pale ale,33 acres of ocean,33 acres,33acres,-1
103,33 acres of sunshine blanche,33 acres of sunshine,33 acres,33acres,-1
104,fernie brewing - the real peel grapefruit ipa ...,grapefruit ipa,ferniebrewing,4mile,-1


In [25]:
shared_name

0                 four winds - nectarous dry hopped sour
1                                                    NaN
2                        driftwood brewery - fat tug ipa
3                                                    NaN
4                                                    NaN
5      driftwood brewery - raised by wolves ipa tall ...
6                                                    NaN
7                        four winds - juxtapose wild ipa
8                                                    NaN
9                                                    NaN
10                                                   NaN
11                                                   NaN
12                                                   NaN
13                                                   NaN
14                                                   NaN
15                                                   NaN
16                                                   NaN
17                   yellow dog

In [26]:
reviews2['comb'] = shared_name

In [27]:
reviews2['mnf_comb'] = dj.mnfcomp

In [28]:
reviews2_to_merge = reviews2[reviews2.mnf_comb==0]

In [29]:
len(reviews2_to_merge.comb)

59

In [30]:
df =pd.merge(lq, reviews2_to_merge, how ='left', left_on = lq.name, right_on = reviews2_to_merge.comb)

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 511 entries, 0 to 510
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   key_0              511 non-null    object 
 1   name_x             511 non-null    object 
 2   country            511 non-null    object 
 3   price              511 non-null    object 
 4   sku_link           511 non-null    object 
 5   alc_x              511 non-null    object 
 6   style_x            511 non-null    object 
 7   beer_type          511 non-null    object 
 8   desc               511 non-null    object 
 9   mnf                64 non-null     object 
 10  name_y             64 non-null     object 
 11  style_y            64 non-null     object 
 12  alc_y              64 non-null     object 
 13  ratings            64 non-null     float64
 14  avg_rating         64 non-null     float64
 15  detail_rating      64 non-null     object 
 16  avg_per_user       64 non-

In [32]:
df.columns

Index(['key_0', 'name_x', 'country', 'price', 'sku_link', 'alc_x', 'style_x',
       'beer_type', 'desc', 'mnf', 'name_y', 'style_y', 'alc_y', 'ratings',
       'avg_rating', 'detail_rating', 'avg_per_user', 'number_of_reviews',
       'text_reviews', 'rev_link', 'comb', 'mnf_comb'],
      dtype='object')

In [43]:
df

,key_0,name_x,country,price,sku_link,alc_x,style_x,beer_type,desc,mnf,name_y,style_y,alc_y,ratings,avg_rating,detail_rating,avg_per_user,number_of_reviews,text_reviews,rev_link,comb,mnf_comb
0,33 acres of life lager,33 acres of life lager,Canada,$12.29,/product/772335,4.8%,California Common Or Steam,Domestic Beer - BC Craft Beer,"1848, West Coast convictions of resourcefulnes...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,33 acres of nirvana 6 pack,33 acres of nirvana 6 pack,Canada,$13.49,/product/880930,7%,India Pale Ale,Domestic Beer - BC Craft Beer,33 Acres is one of Vancouver’s most well-loved...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,33 acres of ocean pale ale,33 acres of ocean pale ale,Canada,$12.29,/product/837591,5.6%,Pale Ale,Domestic Beer - BC Craft Beer,A full flavoured beer integrated with a distin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,33 acres of sunshine blanche,33 acres of sunshine blanche,Canada,$12.29,/product/90837,5%,Wheat,Domestic Beer - BC Craft Beer,"Thanks to summers' launching acclaims, this ro...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,33 acres of sunshine can,33 acres of sunshine can,Canada,$21.99,/product/23859,5%,Wheat,Domestic Beer - BC Craft Beer,No info,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,alexander keith's - india pale ale can,alexander keith's - india pale ale can,Canada,$25.99,/product/588681,5%,India Pale Ale,Domestic Beer - National Domestic Beer,"Brewed in Halifax since 1820, India Pale Ale i...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,alexander keith's - india pale ale can,alexander keith's - india pale ale can,Canada,$14.99,/product/687251,5%,India Pale Ale,Domestic Beer - National Domestic Beer,"Brewed in Halifax since 1820, India Pale Ale i...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,alhambra reserva 1925,alhambra reserva 1925,Spain,$13.29,/product/638759,6.4%,Pilsner,Import Beer,This award-winning lager is brewed over 35 day...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,asahi super dry,asahi super dry,Thailand,$14.79,/product/439950,5%,Pilsner,Import Beer,No info,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,asahi super dry big can,asahi super dry big can,Japan,$14.79,/product/541201,5%,Pilsner,Import Beer,No info,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.name_y

In [ ]:
df.head(20)

### outside BC

In [5]:
outsideBC = pd.read_csv("f3_outsideca.csv")

In [6]:
#choose only needed columns
needed_columns = ['beer','mnf_x','style', 'alc', 'ratings', 'avg_rating','detail_rating','avg_per_user','number_of_reviews','text_reviews']
outsideBC = outsideBC[needed_columns]
outsideBC.head()

,beer,mnf_x,style,alc,ratings,avg_rating,detail_rating,avg_per_user,number_of_reviews,text_reviews
0,Yanjing Beer,Beijing Yanjing Beer Group Corporation,['Lager - European Pale'],['4.7%'],['250'],['2.57'],['look: 3.75 | smell: 3.25 | taste: 3.5 | feel...,"['3.38', '3.83', '3.44', '2.44', '2.55', '2.55...",140,['12oz bottle from Brewers Haven. Very pale go...
1,Hobgoblin Ruby 4.5%,['Wychwood Brewery Company Ltd'],['Bitter - English'],['4.5%'],['0'],['0'],[],[],0,[]
2,Omission Pale Ale (Gluten-Free),Widmer Brothers Brewing Company,['Pale Ale - American'],['5.8%'],['645'],['3.36'],['look: 3.5 | smell: 3.75 | taste: 3.75 | feel...,"['3.81', '3.23', '3.36', '3.3', '3.4', '2.6', ...",128,['From the bottle it pours a surprising amber ...
3,Westmalle Trappist Tripel,Brouwerij Westmalle,['Tripel'],['9.5%'],"['4,782']",['4.33'],['look: 4 | smell: 4.25 | taste: 4.5 | feel: 4...,"['4.31', '4.27', '4.33', '3.43', '4.59', '4.38...",1822,"[""Westmalle Trappist 'Tripel' @ 9.5% , served ..."
4,Wernesgruner,Wernesgrüner Brauerei AG,['Pilsner - German'],['4.9%'],['437'],['3.44'],['look: 3.25 | smell: 3.25 | taste: 3.5 | feel...,"['3.33', '3.92', '4.18', '4.36', '4.14', '3.43...",220,"[""Pours a crystal golden color with a pure whi..."


In [21]:
#clean columns
clean = ['mnf_x', 'style', 'alc', 'ratings', 'avg_rating','detail_rating']
for i in clean:
    outsideBC[i] = outsideBC[i].str.replace('\['|'\]|\["|"\]','')
    
outsideBC['avg_per_user'] = outsideBC[i].str.replace('\[|\]','')

TypeError: unsupported operand type(s) for |: 'str' and 'str'

In [14]:
outsideBC['avg_per_user'] = outsideBC['avg_per_user'].str.replace('\['|'\]|\["|"\]','')

TypeError: unsupported operand type(s) for |: 'str' and 'str'

In [69]:
outsideBC.head()

,beer,mnf_x,style,alc,ratings,avg_rating,detail_rating,avg_per_user,number_of_reviews,text_reviews
0,Yanjing Beer,Beijing Yanjing Beer Group Corporation,Lager - European Pale,4.7%,250,2.57,look: 3.75 | smell: 3.25 | taste: 3.5 | feel: ...,"3.38', '3.83', '3.44', '2.44', '2.55', '2.55',...",140,12oz bottle from Brewers Haven. Very pale gold...
1,Hobgoblin Ruby 4.5%,Wychwood Brewery Company Ltd,Bitter - English,4.5%,0,0,[],[],0,[]
2,Omission Pale Ale (Gluten-Free),Widmer Brothers Brewing Company,Pale Ale - American,5.8%,645,3.36,look: 3.5 | smell: 3.75 | taste: 3.75 | feel: ...,"3.81', '3.23', '3.36', '3.3', '3.4', '2.6', '3...",128,From the bottle it pours a surprising amber an...
3,Westmalle Trappist Tripel,Brouwerij Westmalle,Tripel,9.5%,"4,782",4.33,look: 4 | smell: 4.25 | taste: 4.5 | feel: 4 |...,"4.31', '4.27', '4.33', '3.43', '4.59', '4.38',...",1822,"[""Westmalle Trappist 'Tripel' @ 9.5% , served ..."
4,Wernesgruner,Wernesgrüner Brauerei AG,Pilsner - German,4.9%,437,3.44,look: 3.25 | smell: 3.25 | taste: 3.5 | feel: ...,"3.33', '3.92', '4.18', '4.36', '4.14', '3.43',...",220,"[""Pours a crystal golden color with a pure whi..."
